# Retrieval-Augmented Generation (RAG)

In [13]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

In [14]:
chroma_client = chromadb.PersistentClient("../chroma_Jagan")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [15]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)

[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.
[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.


In [42]:
@function_tool
def calorie_lookup_tool_rag(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [20]:
calorie_lookup_tool_rag('apple')

'Nutrition Information:\nApple (Fruits): 52.0 calories per 100g\nApple Juice ((Fruit)Juices): 46.0 calories per 100g\nApplesauce (Fruits): 68.0 calories per 100g'

In [25]:
chroma_client = chromadb.PersistentClient("../chroma_Jagan")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")

In [26]:
results = nutrition_qna.query(query_texts=["pregnancy"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)

[('answer', 'A pregnant woman might not gain adequate weight due to various causes. These could encompass inadequate nutrition, medical problems such as infections or chronic diseases and deficiencies of certain key nutrients.'), ('answer_length', 213), ('has_question_mark', True), ('keywords', 'a or encompass not during certain increase nutrition diseases gain deficiencies chronic and pregnant might adequate hinder of causes circumstances to s healthy gestation various nutrients problems could weight may medical infections due as woman such inadequate key what these'), ('question', "What circumstances may hinder a woman's healthy weight increase during gestation?"), ('question_length', 81), ('topic', 'nutrition_qa')]
Question: What circumstances may hinder a woman's healthy weight increase during gestation?
        Answer: A pregnant woman might not gain adequate weight due to various causes. These could encompass inadequate nutrition, medical problems such as infections or chronic di

In [41]:
@function_tool
def qna_lookup_tool_rag(query: str, max_results: int = 3) -> str:
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """


    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [44]:
qna_lookup_tool_rag('pregnancy')

TypeError: 'FunctionTool' object is not callable

In [45]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    """,
    tools=[calorie_lookup_tool_rag,qna_lookup_tool_rag]
)

In [46]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool_rag, qna_lookup_tool_rag],
)

In [47]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here’s concise guidance for pregnant women, including typical calorie guidance:

- Focus on nutrient-dense meals:
  - Cereals and millets (whole grains)
  - Dals/eggs/milk/meat/fish
  - Protective vegetables and vitamin C–rich fruits
  - Dark green leafy vegetables
  - Other vegetables and fruits
  - Include iron and folate sources (e.g., fortified cereals, leafy greens, beans, lean meat, dairy)

- Calorie guidance:
  - In the last trimester, aim for about 300 extra calories per day on top of your usual intake.

If you want, I can suggest a sample day’s meals with approximate calorie ranges.


In [39]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "pregnancy",
    )
    print(result.final_output)

AttributeError: 'function' object has no attribute 'name'

In [ ]:
#answer by 

@function_tool
def nutrtition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nuntrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)